In [1]:


from __future__ import annotations
import datetime as dt
import json, math, os, warnings
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import copy


import numpy as np
import pandas as pd
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
except ImportError:
    print("Error: PyTorch not installed. The script cannot run.")
    exit()
    
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler


try:
    import pmdarima as pm
    
    from statsmodels.tsa.exponential_smoothing.ets import ETSModel
except ImportError:
    print("Warning: pmdarima or latest statsmodels not installed. Baselines (ARIMA/ETS) will be skipped.")
    pm = None
    ETSModel = None

from statsmodels.tsa.api import Holt, SimpleExpSmoothing
from statsmodels.tools.sm_exceptions import ConvergenceWarning, ValueWarning
from scipy.stats import t


try:
    import mrmr
except ImportError:
    print("Warning: mrmr-selection not installed. Multivariate mode will fail if used.")
    mrmr = None

try:
    import optuna
    from optuna.samplers import TPESampler
    from optuna.pruners import SuccessiveHalvingPruner
except ImportError:
    print("Error: Optuna not installed. The script cannot run.")
    exit()


ROOT = Path("/yourdatahere") 
BASE_OUT = ROOT / "36_forecast_outputs_lstm_strategy1"
BASE_OUT.mkdir(parents=True, exist_ok=True)
STORAGE = f"sqlite:///{BASE_OUT / 'lstm_rigorous_optuna_study.db'}"


FORECAST_HORIZON = 36 # The target horizon (H)
SEASONAL_PERIOD = 12


OPTUNA_N_TRIALS = 50
MAX_EPOCHS_OPTUNA = 30
PATIENCE_ES = 5 
N_CV_SPLITS = 5 


MAX_EPOCHS_FINAL = 150 
WEIGHT_DECAY = 1e-4
RANDOM_STATE = 42


MC_SAMPLES_FOR_CI = 100
EPSILON = 1e-8


torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


try:
    torch.backends.cudnn.benchmark = True
    if torch.cuda.is_available():
        if hasattr(torch.backends.cuda.matmul, 'allow_tf32'):
             torch.backends.cuda.matmul.allow_tf32 = True
        if hasattr(torch, 'set_float32_matmul_precision'):
            torch.set_float32_matmul_precision("medium")
except Exception:
    pass


class LSTMForecaster(nn.Module):
    """
    LSTM model adapted for Direct Multi-Step (DMS) forecasting.
    """
    def __init__(self, input_size: int, hidden_size: int,
                 output_size: int, dropout: float, num_layers: int):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )
        self.dropout = nn.Dropout(dropout)
        
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        
        out, _ = self.lstm(x)
        
        out = self.dropout(out[:, -1, :])
        
        return self.fc(out)

def build_model(hp: Dict, n_vars: int) -> nn.Module:
    mdl = LSTMForecaster(
        input_size=n_vars,
        hidden_size=int(hp["hidden_size"]),
        output_size=FORECAST_HORIZON, 
        dropout=float(hp["dropout_rate"]),
        num_layers=int(hp["num_layers"]),
    ).to(device)
    try:
        
        if hasattr(torch, 'compile'):
             return torch.compile(mdl, mode="reduce-overhead")
        return mdl
    except Exception:
        return mdl


def smape_loss_np(y_true, y_pred, eps: float = EPSILON) -> float:
    
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    num = np.abs(y_true - y_pred)
    den = (np.abs(y_true) + np.abs(y_pred) + eps) / 2.0
    
    return np.mean(num / den)

def calculate_metrics(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    insample: Optional[np.ndarray] = None,
):
    
    if y_true.ndim == 1 or (y_true.ndim > 1 and y_true.shape[1] == 1):
        y_true = y_true.ravel()
        y_pred = y_pred.ravel()

    smape = smape_loss_np(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + EPSILON))) * 100.0
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    mase = rmsse = np.nan

    
    if insample is not None and len(insample) > SEASONAL_PERIOD:
       
        d = np.mean(np.abs(np.diff(insample, n=SEASONAL_PERIOD)))
        
        mae = np.mean(np.abs(y_true - y_pred))
        mse = np.mean((y_true - y_pred) ** 2)

        mase = mae / (d + EPSILON)
        
        
        denom_rmsse = np.mean((np.diff(insample, n=SEASONAL_PERIOD))**2)
        rmsse = np.sqrt(mse / (denom_rmsse + EPSILON))


    return {"SMAPE": smape, "MAPE": mape, "RMSE": rmse,
            "MASE": mase, "RMSSE": rmsse}


def diebold_mariano_test(actuals: np.ndarray, pred1: np.ndarray, pred2: np.ndarray, horizon: int):
    """
    Implements the Diebold-Mariano Test (with Newey-West HAC estimator) 
    to compare predictive accuracy of two forecasts.
    """
    actuals = actuals.ravel()
    pred1 = pred1.ravel()
    pred2 = pred2.ravel()

    if len(actuals) != len(pred1) or len(actuals) != len(pred2):
        return np.nan, np.nan
    
    N = len(actuals)
    if N == 0:
        return np.nan, np.nan

    
    loss1 = (actuals - pred1)**2
    loss2 = (actuals - pred2)**2
    d = loss1 - loss2
    d_mean = np.mean(d)

    
    q = horizon 
    gamma = np.zeros(q + 1)
    
   
    for k in range(q + 1):
        if k == 0:
            gamma[k] = np.var(d, ddof=0)
        else:
            
            try:
                cov = np.cov(d[k:], d[:N-k], ddof=0)
                if cov.ndim == 2:
                    gamma[k] = cov[0, 1]
                else:
                    
                    gamma[k] = 0
            except Exception:
                gamma[k] = 0


    
    V = gamma[0] + 2 * np.sum([(1 - k/(q+1)) * gamma[k] for k in range(1, q+1)])

    if V <= 1e-9:
        
        if abs(d_mean) < 1e-9:
            return 0, 1.0 # Forecasts are identical
        else:
            return np.inf * np.sign(d_mean), 0.0

    
    DM_stat = d_mean / np.sqrt(V / N)

    
    p_value = 2 * (1 - t.cdf(np.abs(DM_stat), df=N-1))

    return DM_stat, p_value


def fit_smoother(series, method, alpha=None, beta=None):
    if method == "NS":
        return None
    try:
        if method == "ES":
            return SimpleExpSmoothing(series, initialization_method="estimated").fit(
                smoothing_level=alpha, optimized=False)
        if method == "DES":
            return Holt(series, initialization_method="estimated").fit(
                smoothing_level=alpha, smoothing_trend=beta, optimized=False)
    except Exception:
        
        return None
    raise ValueError(f"Unknown smoothing method: {method}")

def apply_fitted_smoother(fitter, df: pd.DataFrame, target: str) -> pd.DataFrame:
    if fitter is None:
        return df
    n = len(df)
    fitted_vals = fitter.fittedvalues
    
    if len(fitted_vals) == 0:
        return df

    if len(fitted_vals) == n:
        values = fitted_vals
    elif len(fitted_vals) < n:
        
        values = pd.concat([fitted_vals, df[target].iloc[len(fitted_vals):]])
    else:
        values = fitted_vals[:n]

    out = df.copy()
    out[target] = values.values
    #
    out[target].bfill(inplace=True)
    return out


def create_sequences_dms(df_in: pd.DataFrame, target: str,
                         features: List[str], num_lags: int, horizon: int) -> Tuple[np.ndarray, np.ndarray, List[str]]:
    """
    Creates input sequences (X) and target sequences (Y) for Direct Multi-Step forecasting.
    
    X: (N_samples, num_lags, N_features)
    Y: (N_samples, horizon)
    """
    cols = [target] + features
    data = df_in[cols].values
    N = data.shape[0]

    X, Y = [], []

    # Iterate through the data to create sequences
    for i in range(N - num_lags - horizon + 1):
        # Input sequence (X): from index i to i + num_lags
        x_seq = data[i : i + num_lags]
        # Target sequence (Y): from index i + num_lags to i + num_lags + horizon
        # We only want the target variable (index 0) for Y
        y_seq = data[i + num_lags : i + num_lags + horizon, 0]

        X.append(x_seq)
        Y.append(y_seq)

    return np.array(X), np.array(Y), cols


def select_mrmr_features(df_window: pd.DataFrame, target: str, k: int, num_lags: int) -> List[str]:
    """
    Selects features using mRMR based on the relationship with the immediate next target value (t+1).
    """
    if k == 0 or mrmr is None:
        return []

    # 1. Create temporary lagged dataframe for mRMR selection
    lagged = []
    potential_features = [c for c in df_window.columns if c != target]
    if not potential_features:
        return []

    # Create lags
    for col in [target] + potential_features:
        for l in range(1, num_lags + 1):
            lagged.append(df_window[col].shift(l).rename(f"{col}_lag_{l}"))

    df_lagged = pd.concat([df_window[[target]], *lagged], axis=1).dropna()

    if df_lagged.empty:
        return []

    # 2. Apply mRMR
    # Select the best *exogenous* features (excluding target's own lags from the selection pool)
    X_cols = [c for c in df_lagged.columns if "_lag_" in c and not c.startswith(f"{target}_lag_")]

    if not X_cols:
        return []

    X = df_lagged[X_cols].ffill().fillna(0)
    y = df_lagged[target].values.ravel()

    # Limit K to the number of available features
    K_eff = min(k, len(X_cols))

    try:
        # Use mrmr_regression for continuous target
        selected_lagged = mrmr.mrmr_regression(X=X, y=y, K=K_eff)
        # Get the base feature names
        selected_base = list({s.split("_lag_")[0] for s in selected_lagged})
        return selected_base
    except Exception as e:
        print(f"Warning: mRMR failed: {e}. Falling back to using no exogenous features.")
        return []

# ---------- Prediction Utilities ----------
def mc_pred(model: nn.Module, inp: torch.Tensor, mc_samples: int):
    """Generates Monte Carlo predictions if dropout is active."""
    # inp shape: (1, Lags, Features)
    if mc_samples == 1:
        model.eval() # Ensure eval mode if not doing MC dropout
        with torch.no_grad():
            # Output shape: (1, Horizon)
            return model(inp).cpu().numpy()

    # Enable dropout at inference time (MC Dropout)
    model.train() 
    with torch.no_grad():
        # Repeat input MC times. Shape: (MC_samples, Lags, Features)
        inp_mc = inp.repeat(mc_samples, 1, 1)
        # Output shape: (MC_samples, Horizon)
        preds = model(inp_mc).cpu().numpy()
    return preds


# =========================================================
#                 3.  Optuna objective (Updated for DMS)
# =========================================================
# Addressed Weakness 4: HPO now optimizes the 36-month forecast SMAPE.
def make_objective(tgt: str, train_val_df: pd.DataFrame,
                   k: int, mode: str):
    # Use TimeSeriesSplit for HPO validation. The test_size is the forecast horizon.
    tscv = TimeSeriesSplit(n_splits=N_CV_SPLITS, test_size=FORECAST_HORIZON)

    def objective(trial: optuna.Trial):
        hp = {
            # Architecture HPs (Expanded ranges)
            "hidden_size": trial.suggest_categorical("hidden_size", [64, 128, 256, 512]),
            "num_layers": trial.suggest_int("num_layers", 1, 3),
            "dropout_rate": trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1),
            # Training HPs
            "lr": trial.suggest_categorical("lr", [1e-4, 5e-4, 1e-3]),
            "batch_size": trial.suggest_categorical("batch_size", [32, 64, 128]),
            # Feature Engineering HPs (Crucial for sequence length)
            "n_lags": trial.suggest_int("n_lags", SEASONAL_PERIOD, SEASONAL_PERIOD * 4, step=SEASONAL_PERIOD),
            # Optional Preprocessing (Smoothing)
            "smoothing_method": trial.suggest_categorical("smoothing_method", ["NS", "DES"]),
            "smoothing_alpha": trial.suggest_float("smoothing_alpha", 0.1, 0.5),
            "smoothing_beta": trial.suggest_float("smoothing_beta", 0.05, 0.3),
        }

        n_lags = hp["n_lags"]
        smapes_cv = []

        # Iterate over Cross-Validation folds
        for tr_idx, val_idx in tscv.split(train_val_df):
            # Ensure enough data in training fold for lags and at least one training sample
            if len(tr_idx) < n_lags + FORECAST_HORIZON:
                continue

            df_tr = train_val_df.iloc[tr_idx]
            
            # 1. Preprocessing (Smoothing) - Fit only on training data
            smoother = fit_smoother(df_tr[tgt],
                                    hp["smoothing_method"],
                                    hp["smoothing_alpha"],
                                    hp["smoothing_beta"])
            df_tr_sm = apply_fitted_smoother(smoother, df_tr, tgt)

            # 2. Feature Selection (mRMR)
            if mode == "MULTI":
                # Select features based on the smoothed training data
                # Use a fixed lag length (e.g., 12) for the mRMR selection process itself
                base_feats = select_mrmr_features(df_tr_sm, tgt, k, num_lags=SEASONAL_PERIOD) 
            else:
                base_feats = []

            n_vars = len([tgt, *base_feats])

            # 3. Data Sequencing (DMS format)
            X_tr, Y_tr, _ = create_sequences_dms(df_tr_sm, tgt, base_feats, n_lags, FORECAST_HORIZON)

            # Prepare the validation input: The very last 'n_lags' of the training fold
            X_va_input = df_tr_sm[[tgt] + base_feats].iloc[-n_lags:].values.reshape(1, n_lags, n_vars)
            # Validation target: The actual 'FORECAST_HORIZON' points in the validation fold (ground truth)
            Y_va_target = train_val_df.iloc[val_idx][tgt].values.reshape(1, FORECAST_HORIZON)

            if X_tr.shape[0] == 0:
                continue # Not enough data for this fold/configuration

            # 4. Scaling
            # Reshape for MinMaxScaler
            X_tr_reshaped = X_tr.reshape(-1, n_vars)
            Y_tr_reshaped = Y_tr.reshape(-1, 1)

            sc_X = MinMaxScaler().fit(X_tr_reshaped)
            sc_y = MinMaxScaler().fit(Y_tr_reshaped)

            # Apply scaling and reshape back
            X_tr_scaled = sc_X.transform(X_tr.reshape(-1, n_vars)).reshape(X_tr.shape)
            Y_tr_scaled = sc_y.transform(Y_tr.reshape(-1, 1)).reshape(Y_tr.shape)
            X_va_scaled = sc_X.transform(X_va_input.reshape(-1, n_vars)).reshape(X_va_input.shape)

            # 5. Tensors and DataLoader
            Xtr_t = torch.tensor(X_tr_scaled, dtype=torch.float32)
            Ytr_t = torch.tensor(Y_tr_scaled, dtype=torch.float32)
            Xva_t = torch.tensor(X_va_scaled, dtype=torch.float32).to(device)

            dl = torch.utils.data.DataLoader(
                torch.utils.data.TensorDataset(Xtr_t, Ytr_t),
                batch_size=hp["batch_size"], shuffle=True)

            # 6. Model Training
            mdl = build_model(hp, n_vars)
            # L1Loss (MAE) is often robust for time series
            crit = nn.L1Loss() 
            opt = optim.AdamW(
                mdl.parameters(), lr=hp["lr"],
                weight_decay=WEIGHT_DECAY)

            best_fold_smape = float("inf")
            no_imp = 0

            for ep in range(MAX_EPOCHS_OPTUNA):
                mdl.train()
                for xb, yb in dl:
                    xb, yb = xb.to(device), yb.to(device)
                    opt.zero_grad(set_to_none=True)
                    # Mixed precision training
                    with torch.autocast(device.type, enabled=device.type == "cuda"):
                        preds = mdl(xb)
                        # Loss is calculated over all H steps
                        loss = crit(preds, yb) 
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(mdl.parameters(), 1.0)
                    opt.step()

                # 7. Validation (Crucial: Evaluate on the H-step forecast)
                mdl.eval()
                with torch.no_grad():
                    # Generate the H-step forecast
                    preds_scaled = mdl(Xva_t).cpu().numpy() # Shape (1, Horizon)
                    # Inverse transform
                    preds_inv = sc_y.inverse_transform(preds_scaled.reshape(-1, 1)).reshape(1, FORECAST_HORIZON)

                    # Calculate SMAPE against the ground truth validation target
                    s = smape_loss_np(Y_va_target, preds_inv)

                # Optuna pruning and Early Stopping logic
                trial.report(s, ep)
                if trial.should_prune():
                    raise optuna.TrialPruned()

                if s < best_fold_smape - 1e-4:
                    best_fold_smape = s
                    no_imp = 0
                else:
                    no_imp += 1

                if no_imp >= PATIENCE_ES:
                    break

            smapes_cv.append(best_fold_smape)

        if not smapes_cv:
            # If no folds completed (e.g., data too short for lags)
            return float("inf")

        # Return the average SMAPE across CV folds
        return float(np.mean(smapes_cv))

    return objective

# =========================================================
#            4. HPO and Training Pipeline (Reworked)
# =========================================================
# Addressed Weakness 6 & 8: HPO via CV, Final training uses Early Stopping.

def train_final_model(hp: Dict, df_train: pd.DataFrame, tgt: str, base_feats: List[str], use_early_stopping: bool = True):
    """
    Trains the LSTM model with optimized hyperparameters.
    Includes early stopping on a validation subset of the training data.
    """
    n_lags = hp["n_lags"]
    n_vars = len([tgt, *base_feats])

    # 1. Preprocessing (Smoothing)
    smoother = fit_smoother(df_train[tgt],
                            hp["smoothing_method"],
                            hp["smoothing_alpha"],
                            hp["smoothing_beta"])
    df_train_sm = apply_fitted_smoother(smoother, df_train, tgt)

    # 2. Data Sequencing
    X, Y, _ = create_sequences_dms(df_train_sm, tgt, base_feats, n_lags, FORECAST_HORIZON)

    if X.shape[0] == 0:
        return None, None, None

    # 3. Splitting for Early Stopping
    # We reserve the most recent 20% of sequences for validation if ES is enabled.
    if use_early_stopping and X.shape[0] > 20:
         split_idx = int(X.shape[0] * 0.8)
         X_tr, Y_tr = X[:split_idx], Y[:split_idx]
         X_va, Y_va = X[split_idx:], Y[split_idx:]
    else:
        X_tr, Y_tr = X, Y
        X_va, Y_va = None, None

    # 4. Scaling (Fit only on the training split)
    X_tr_reshaped = X_tr.reshape(-1, n_vars)
    Y_tr_reshaped = Y_tr.reshape(-1, 1)

    sc_X = MinMaxScaler().fit(X_tr_reshaped)
    sc_y = MinMaxScaler().fit(Y_tr_reshaped)

    X_tr_scaled = sc_X.transform(X_tr.reshape(-1, n_vars)).reshape(X_tr.shape)
    Y_tr_scaled = sc_y.transform(Y_tr.reshape(-1, 1)).reshape(Y_tr.shape)

    Xtr_t = torch.tensor(X_tr_scaled, dtype=torch.float32)
    Ytr_t = torch.tensor(Y_tr_scaled, dtype=torch.float32)

    if X_va is not None:
        X_va_scaled = sc_X.transform(X_va.reshape(-1, n_vars)).reshape(X_va.shape)
        Y_va_scaled = sc_y.transform(Y_va.reshape(-1, 1)).reshape(Y_va.shape)
        Xva_t = torch.tensor(X_va_scaled, dtype=torch.float32).to(device)
        Yva_t = torch.tensor(Y_va_scaled, dtype=torch.float32).to(device)
    else:
        Xva_t, Yva_t = None, None

    dl = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(Xtr_t, Ytr_t),
        batch_size=hp["batch_size"], shuffle=True)

    # 5. Model Initialization
    mdl = build_model(hp, n_vars)
    crit = nn.L1Loss()
    opt = optim.AdamW(
        mdl.parameters(), lr=hp["lr"],
        weight_decay=WEIGHT_DECAY)

    # 6. Training Loop with Early Stopping
    best_val_loss = float("inf")
    no_imp = 0
    best_model_state = None

    for ep in range(MAX_EPOCHS_FINAL):
        mdl.train()
        for xb, yb in dl:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad(set_to_none=True)
            with torch.autocast(device.type, enabled=device.type == "cuda"):
                preds = mdl(xb)
                loss = crit(preds, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(mdl.parameters(), 1.0)
            opt.step()

        # Validation phase for Early Stopping
        if Xva_t is not None:
            mdl.eval()
            with torch.no_grad():
                 with torch.autocast(device.type, enabled=device.type == "cuda"):
                    val_preds = mdl(Xva_t)
                    val_loss = crit(val_preds, Yva_t).item()

            if val_loss < best_val_loss - 1e-4:
                best_val_loss = val_loss
                no_imp = 0
                # Save best model weights
                best_model_state = copy.deepcopy(mdl.state_dict())
            else:
                no_imp += 1

            if no_imp >= PATIENCE_ES:
                # print(f"Early stopping triggered at epoch {ep+1}/{MAX_EPOCHS_FINAL}")
                break
    
    # Load the best weights found during training
    if best_model_state:
         mdl.load_state_dict(best_model_state)

    return mdl, sc_X, sc_y


def run_hpo_pipeline(df_dev: pd.DataFrame, targets: List[str], k: int, mode: str) -> Dict:
    """
    Runs the Hyperparameter Optimization pipeline for all targets on the development set.
    """
    best_cfgs = {}
    print(f"\n--- Starting HPO Pipeline (Mode: {mode}) ---")

    for tgt in targets:
        print(f"\nOptimizing Target: {tgt}")

        # Define the study
        study_name = f"{mode}_{tgt}_HPO_DMS"
        try:
            study = optuna.create_study(
                study_name=study_name,
                direction="minimize", # Minimize H-month SMAPE
                sampler=TPESampler(seed=RANDOM_STATE),
                pruner=SuccessiveHalvingPruner(min_resource=4, reduction_factor=3),
                storage=STORAGE, load_if_exists=True)
        except Exception as e:
             print(f"Could not create/load Optuna study: {e}")
             continue

        # Run optimization
        if len(study.trials) < OPTUNA_N_TRIALS:
            objective_fn = make_objective(tgt, df_dev, k, mode)
            study.optimize(objective_fn,
                           n_trials=OPTUNA_N_TRIALS - len(study.trials),
                           n_jobs=1, # Keep n_jobs=1 for stability with PyTorch/GPU
                           show_progress_bar=True)

        if study.best_trial is None:
            print(f"HPO failed for {tgt}. Skipping.")
            continue

        print(f"Best CV SMAPE for {tgt}: {study.best_value:.4f}")
        
        best_hp = study.best_params
        
        # Determine the final feature set by running mRMR on the full development set
        # using the optimized preprocessing parameters.
        if mode == "MULTI":
            smoother = fit_smoother(df_dev[tgt],
                                    best_hp.get("smoothing_method", "NS"),
                                    best_hp.get("smoothing_alpha", 0.5),
                                    best_hp.get("smoothing_beta", 0.1))
            df_dev_sm = apply_fitted_smoother(smoother, df_dev, tgt)
            # Use a standard lag (e.g., 12) for the feature selection process
            final_feats = select_mrmr_features(df_dev_sm, tgt, k, num_lags=SEASONAL_PERIOD)
        else:
            final_feats = []

        best_cfgs[tgt] = {
            "smape_dev": study.best_value,
            "hyperparams": best_hp,
            "base_features": final_feats,
            "n_vars_per_step": len([tgt, *final_feats]),
            "n_lags": best_hp["n_lags"]
        }

    return best_cfgs

# =========================================================
#                 5.  Forecast & Evaluation
# =========================================================

def generate_forecast_dms(
    df_hist: pd.DataFrame, target: str, cfg: Dict, horizon: int,
    mc_samples: int):
    """
    Trains the model on historical data and generates a H-step ahead forecast with CI.
    """
    if not cfg or not cfg.get("hyperparams"):
        return None

    hp = cfg["hyperparams"]
    feats = cfg["base_features"]
    n_lags = cfg["n_lags"]
    n_vars = cfg["n_vars_per_step"]

    if horizon != FORECAST_HORIZON:
         raise ValueError(f"Model architecture fixed to horizon {FORECAST_HORIZON}, cannot forecast {horizon}.")

    # 1. Train the model on the full historical data (df_hist)
    mdl, sc_X, sc_y = train_final_model(hp, df_hist, target, feats, use_early_stopping=True)

    if mdl is None:
        return None

    # 2. Prepare the input for forecasting (the last 'n_lags' points)

    # We must apply the same preprocessing (smoothing) to the historical data 
    # before creating the final input sequence.
    smoother = fit_smoother(df_hist[target],
                            hp["smoothing_method"],
                            hp["smoothing_alpha"],
                            hp["smoothing_beta"])
    df_hist_sm = apply_fitted_smoother(smoother, df_hist, target)

    input_data = df_hist_sm[[target] + feats].iloc[-n_lags:].values
    
    # Scale the input
    input_scaled = sc_X.transform(input_data.reshape(-1, n_vars))
    # Reshape for model input: (1, Lags, Features)
    inp_t = torch.tensor(input_scaled.reshape(1, n_lags, n_vars),
                         dtype=torch.float32, device=device)

    # 3. Generate Forecasts (with MC Dropout for Uncertainty)
    # Use MC samples if requested AND if the model has dropout > 0.
    use_mc = mc_samples > 1 and hp.get("dropout_rate", 0) > 0
    samples = mc_samples if use_mc else 1

    # preds_mc shape: (Samples, Horizon)
    preds_mc_scaled = mc_pred(mdl, inp_t, samples)

    # Inverse transform
    preds_inv = sc_y.inverse_transform(preds_mc_scaled.reshape(-1, 1)).reshape(samples, horizon)

    # 4. Calculate point forecast and CIs (95% CI)
    if use_mc:
        # Use median as the point forecast (robust)
        forecasts = np.median(preds_inv, axis=0)
        lowers = np.percentile(preds_inv, 2.5, axis=0)
        uppers = np.percentile(preds_inv, 97.5, axis=0)
    else:
        forecasts = preds_inv[0]
        # CIs are NaN if MC Dropout is not used
        lowers = np.full(horizon, np.nan)
        uppers = np.full(horizon, np.nan)

    # Optional: Ensure non-negative forecasts if the domain requires it
    forecasts[forecasts < 0] = 0
    if use_mc:
        lowers[lowers < 0] = 0
        uppers[uppers < 0] = 0

    return pd.DataFrame({"Forecast": forecasts,
                         "Lower_CI": lowers, "Upper_CI": uppers})

# ---------- Baseline Models (Weakness 2 Addressed) ----------

def forecast_arima(series: pd.Series, horizon: int):
    if pm is None:
        return np.full(horizon, np.nan)
    try:
        # Rigorous Auto-ARIMA implementation
        model = pm.auto_arima(series,
                              start_p=1, start_q=1,
                              max_p=5, max_q=5,
                              m=SEASONAL_PERIOD,
                              start_P=0, seasonal=True,
                              d=None, D=1, trace=False,
                              error_action='ignore',  
                              suppress_warnings=True, 
                              stepwise=True)
        forecast = model.predict(n_periods=horizon)
        return np.asarray(forecast)
    except Exception:
        # ARIMA can fail if the series is constant or too short
        return np.full(horizon, np.nan)

def forecast_ets(series: pd.Series, horizon: int):
    if ETSModel is None:
        return np.full(horizon, np.nan)
    try:
        # Optimized ETS implementation (Automatic model selection)
        # We try additive components first as they are generally more robust.
        model = ETSModel(series, error="add", trend="add", seasonal="add",
                         damped_trend=True, seasonal_periods=SEASONAL_PERIOD)
        fit = model.fit(disp=False, optimized=True)
        forecast = fit.forecast(horizon)
        return np.asarray(forecast)
    except Exception:
        return np.full(horizon, np.nan)


# =========================================================
#                 6.  Main Execution
# =========================================================
if __name__ == "__main__":
    # Set appropriate warning levels
    warnings.simplefilter("ignore", ConvergenceWarning)
    warnings.simplefilter("ignore", ValueWarning)
    warnings.simplefilter("ignore", UserWarning)
    # Reduce verbosity for cleaner output during HPO
    if optuna:
        optuna.logging.set_verbosity(optuna.logging.WARNING)

    # --- Environment Logging ---
    env_info = {
        "timestamp_utc": dt.datetime.utcnow().isoformat() + "Z",
        "torch": torch.__version__ if torch else "N/A",
        "FORECAST_HORIZON": FORECAST_HORIZON,
    }
    if torch and torch.cuda.is_available():
        env_info["device"] = torch.cuda.get_device_name(0)
    else:
        env_info["device"] = "CPU"
        
    try:
        json.dump(env_info, open(BASE_OUT / "env.json", "w"), indent=2)
    except Exception as e:
        print(f"Could not write env.json: {e}")

    # ------------- Load & Clean Data -------------
    DF_PATH = GDRIVE_ROOT / "quantum computing updated data excel.xlsx"
    
    # !!! DATA LOADING !!!
    # Replace this MOCK DATA section with your actual data loading if running locally:
    if DF_PATH.exists():
        df_raw = pd.read_excel(DF_PATH)
    else: 
      print("Data file not found. Proceeding with MOCK DATA.")
    



    date_col = next((c for c in df_raw.columns
                     if c.lower().strip() == "date"), None)
    if date_col:
        df_raw[date_col] = pd.to_datetime(df_raw[date_col])
        df_raw.sort_values(date_col, inplace=True)
        df_raw.set_index(date_col, inplace=True)

    df_raw.columns = df_raw.columns.str.lower().str.strip()
    
    # Data Cleaning (Numeric conversion and imputation)
    for col in df_raw.columns:
        if df_raw[col].dtype == "object":
            df_raw[col] = pd.to_numeric(
                df_raw[col].str.replace(r"[^\d.\-]", "", regex=True),
                errors="coerce")
    # Imputation: ffill is reasonable. Filling initial NaNs with 0 must be justified by the domain.
    df_raw = df_raw.ffill().fillna(0)

    # ------------- Data Splitting -------------
    # Rigorous split: Development (Train+Val) and Test (Hold-out)
    # The test set size MUST equal the forecast horizon.
    df_dev = df_raw.iloc[:-FORECAST_HORIZON].copy()
    df_test = df_raw.iloc[-FORECAST_HORIZON:].copy()

    if len(df_dev) < FORECAST_HORIZON * 3:
         print("Warning: Development set might be short relative to the horizon, potentially impacting CV stability.")

    # Define targets
    targets_all = [
        "quantum computing journal article", "generative ai journal article",
        "llm journal article", "autonomous vehicles journal article",
        "digital twins journal article", "next generation ai journal article",
        "advances in cybersecurity journal article", "misinformation journal article",
        "3d printing journal article", "new programming models journal article",
        "reliability journal article", "renewable energy journal article",
        "sustainable technologies journal article", "generative agritech journal article",
        "metaverse journal article",
    ]
    # Filter targets present in the dataframe
    targets = [t.lower().strip() for t in targets_all
               if t.lower().strip() in df_dev.columns]
    
    if not targets:
        # Fallback for mock data if the exact names aren't used
        targets = [col for col in df_raw.columns if 'journal article' in col]
        if not targets:
             raise RuntimeError("No valid targets found in data.")

    # ------------- HPO Execution -------------
    K_FEATURES = 10 # Max exogenous features for mRMR
    best_cfgs_modes = {}

    # Run Univariate and Multivariate pipelines
    for MODE in ["UNI", "MULTI"]:
        if MODE == "MULTI" and (mrmr is None or len(df_raw.columns) < 2):
            print(f"Skipping {MODE} mode (mRMR missing or insufficient features).")
            continue
            
        # Run HPO
        best_cfgs = run_hpo_pipeline(df_dev, targets, k=K_FEATURES, mode=MODE)
        best_cfgs_modes[MODE] = best_cfgs

        # Save HPO results
        best_path = BASE_OUT / f"best_hyperparameters_{MODE}.json"
        try:
            # Atomic write
            tmp = best_path.with_suffix(".tmp")
            with open(tmp, 'w') as f:
                json.dump(best_cfgs, f, indent=2)
            tmp.replace(best_path)
        except Exception as e:
            print(f"Could not save HPO results for {MODE}: {e}")

    # ------------- Test Set Evaluation and Baseline Comparison -------------
    print("\n--- Starting Test Set (Hold-out) Evaluation ---")
    test_results = []
    baseline_cache = {} # Cache baselines as they are the same for UNI/MULTI

    for MODE in best_cfgs_modes.keys():
        best_cfgs = best_cfgs_modes[MODE]

        for tgt in targets:
            print(f"\nEvaluating {MODE} LSTM for: {tgt}")
            cfg = best_cfgs.get(tgt)
            if not cfg:
                continue

            # 1. LSTM Forecast on Test Set
            # Train on df_dev, predict the horizon corresponding to df_test
            # Use mc_samples=1 for point forecast evaluation metrics
            fc_df = generate_forecast_dms(
                df_dev, tgt, cfg, FORECAST_HORIZON, mc_samples=1)

            if fc_df is None:
                print(f"Skipping {tgt} due to model training failure.")
                continue

            y_true = df_test[tgt].values.ravel()
            y_pred_lstm = fc_df["Forecast"].values.ravel()
            insample_data = df_dev[tgt].values

            lstm_m = calculate_metrics(y_true, y_pred_lstm, insample_data)
            result_row = {
                "Target": tgt, "Mode": MODE,
                **{f"LSTM_{k}": v for k, v in lstm_m.items()}
            }

            # 2. Baselines (Weakness 2 Addressed)
            if tgt not in baseline_cache:
                print(f"Calculating Baselines for {tgt} (Naive, SNaive, ARIMA, ETS)...")
                
                # Naive (Persistence)
                naive_pred = np.repeat(insample_data[-1], FORECAST_HORIZON)
                
                # Seasonal Naive
                if len(insample_data) >= SEASONAL_PERIOD:
                    snaive_pred = np.tile(insample_data[-SEASONAL_PERIOD:],
                                          math.ceil(FORECAST_HORIZON/SEASONAL_PERIOD))[:FORECAST_HORIZON]
                else:
                    snaive_pred = naive_pred # Fallback

                # Auto-ARIMA
                arima_pred = forecast_arima(df_dev[tgt], FORECAST_HORIZON)

                # ETS
                ets_pred = forecast_ets(df_dev[tgt], FORECAST_HORIZON)

                baseline_cache[tgt] = {
                    "Naive": naive_pred,
                    "SNaive": snaive_pred,
                    "ARIMA": arima_pred,
                    "ETS": ets_pred,
                }

            # Calculate metrics for baselines and add to results
            for name, pred in baseline_cache[tgt].items():
                 metrics = calculate_metrics(y_true, pred, insample_data)
                 result_row.update({f"{name}_{k}": v for k, v in metrics.items()})

            # 3. Statistical Significance (Weakness 5 Addressed)
            # Compare LSTM against the best statistical model (ARIMA or ETS)
            
            arima_smape = result_row.get("ARIMA_SMAPE", np.inf)
            ets_smape = result_row.get("ETS_SMAPE", np.inf)

            # Determine the best performing statistical baseline
            if (not np.isnan(arima_smape)) and (arima_smape <= ets_smape or np.isnan(ets_smape)):
                best_stat_name = "ARIMA"
                best_stat_pred = baseline_cache[tgt]["ARIMA"]
            elif not np.isnan(ets_smape):
                best_stat_name = "ETS"
                best_stat_pred = baseline_cache[tgt]["ETS"]
            else:
                best_stat_name = None

            if best_stat_name:
                # Check if predictions are valid (non-NaN) before running the test
                if np.isnan(y_pred_lstm).any() or np.isnan(best_stat_pred).any():
                    dm_stat, dm_p = np.nan, np.nan
                else:
                    # Run Diebold-Mariano test
                    dm_stat, dm_p = diebold_mariano_test(y_true, y_pred_lstm, best_stat_pred, FORECAST_HORIZON)

                result_row[f"DM_vs_{best_stat_name}_Stat"] = dm_stat
                result_row[f"DM_vs_{best_stat_name}_Pval"] = dm_p

            test_results.append(result_row)

    # Consolidate and save test results
    df_results = pd.DataFrame(test_results)
    
    # Restructure the results table for easier comparison (One row per target, columns for models)
    if not df_results.empty:
        try:
            if len(df_results["Mode"].unique()) > 1:
                modes = list(df_results["Mode"].unique())
                
                # Assume the first mode calculated the baselines
                df_base = df_results[df_results["Mode"] == modes[0]].drop(columns="Mode")
                
                for mode in modes[1:]:
                    df_mode = df_results[df_results["Mode"] == mode]
                    # Keep only LSTM metrics and DM test results for subsequent modes
                    lstm_cols = [col for col in df_mode.columns if col.startswith("LSTM_") or col.startswith("DM_") or col == "Target"]
                    df_mode = df_mode[lstm_cols]
                    # Rename columns to indicate the mode (e.g., LSTM_SMAPE -> LSTM_MULTI_SMAPE)
                    rename_dict = {c: c.replace("LSTM_", f"LSTM_{mode}_") for c in lstm_cols if c.startswith("LSTM_")}
                    rename_dict.update({c: c.replace("DM_", f"DM_{mode}_") for c in lstm_cols if c.startswith("DM_")})
                    df_mode = df_mode.rename(columns=rename_dict, inplace=False)
                    
                    df_base = pd.merge(df_base, df_mode, on="Target", how="outer")
                
                df_final_results = df_base
            else:
                df_final_results = df_results

            df_final_results.to_csv(BASE_OUT / "test_evaluation_summary.csv", index=False)

        except Exception as e:
            print(f"Error restructuring results table: {e}")
            df_results.to_csv(BASE_OUT / "test_evaluation_raw.csv", index=False)


    # ------------- Long-Term Forecast (Future Projection) -------------
    print("\n--- Generating Final 36-Month Future Forecasts (using full data) ---")
    # Train on the full dataset (df_raw) and forecast the subsequent 36 months.
    df_full = pd.concat([df_dev, df_test])

    for MODE, best_cfgs in best_cfgs_modes.items():
        for tgt in targets:
            cfg = best_cfgs.get(tgt)
            if not cfg:
                continue
                
            print(f"Projecting Future for {tgt} ({MODE})...")

            # Generate forecast with Confidence Intervals using MC Dropout
            fc_df = generate_forecast_dms(
                df_full, tgt, cfg, FORECAST_HORIZON,
                MC_SAMPLES_FOR_CI)

            if fc_df is None:
                continue

            # Save the forecast
            safe_name = "".join(c if c.isalnum() else "_" for c in tgt)
            fc_df.to_csv(BASE_OUT / f"forecast_future_36m_{MODE.lower()}_{safe_name}.csv", index=False)

    print(f"\nAll tasks finished successfully. Outputs are located at: {BASE_OUT}")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Rigorous LSTM (Seq2Seq/DMS) forecasting pipeline for academic research.
Updated: 2025-08-04
"""

from __future__ import annotations
import datetime as dt
import json, math, os, warnings
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import copy

# Required Libraries
import numpy as np
import pandas as pd
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
except ImportError:
    print("Error: PyTorch not installed. The script cannot run.")
    exit()
    
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import t

# --- Imports for rigorous baselines and stats ---
# NOTE: Requires pmdarima, statsmodels>=0.12.0, scipy, mrmr-selection

# Initialize variables to None
pm = None
ETSModel = None
Holt = None
SimpleExpSmoothing = None
ConvergenceWarning = Warning
ValueWarning = Warning

try:
    # Try importing statsmodels components
    from statsmodels.tsa.exponential_smoothing.ets import ETSModel
    from statsmodels.tsa.api import Holt, SimpleExpSmoothing
    from statsmodels.tools.sm_exceptions import ConvergenceWarning, ValueWarning
except ImportError:
    print("Warning: statsmodels not fully installed or outdated. ETS/Smoothing baselines may be skipped.")

try:
    import pmdarima as pm

# Catch the specific ValueError related to NumPy binary incompatibility, and ImportError
except (ImportError, ValueError) as e:
    print(f"\nWarning: Failed to import pmdarima. Auto-ARIMA baseline will be skipped.")
    print(f"Reason: {e}")
    if "numpy.dtype size changed" in str(e):
        print("\nACTION REQUIRED: This is due to NumPy binary incompatibility.")
        print("To fix, run in your terminal: pip install --upgrade numpy && pip install --upgrade --force-reinstall scipy statsmodels pmdarima\n")
    # Ensure they remain None if import failed
    pm = None


# Assuming mrmr is installed (e.g., via pip install mrmr-selection)
mrmr = None
try:
    import mrmr
except ImportError:
    print("Warning: mrmr-selection not installed. Multivariate mode will fail if used.")
    mrmr = None

try:
    import optuna
    from optuna.samplers import TPESampler
    from optuna.pruners import SuccessiveHalvingPruner
except ImportError:
    print("Error: Optuna not installed. The script cannot run.")
    exit()

# ---------- Configuration ----------
# Set your data root directory here
GDRIVE_ROOT = Path("/content") 
BASE_OUT = GDRIVE_ROOT / "36_v4_allsmoothing_forecast_outputs_lstm_rigorous"
BASE_OUT.mkdir(parents=True, exist_ok=True)
STORAGE = f"sqlite:///{BASE_OUT / 'lstm_rigorous_optuna_study.db'}"

# --- Key Parameters ---
FORECAST_HORIZON = 36 # The target horizon (H)
SEASONAL_PERIOD = 12

# --- HPO Parameters ---
OPTUNA_N_TRIALS = 50
MAX_EPOCHS_OPTUNA = 30
PATIENCE_ES = 5 # Patience for Early Stopping
N_CV_SPLITS = 5 # Number of folds for Time Series CV during HPO

# --- Training Parameters ---
MAX_EPOCHS_FINAL = 150 # Max epochs for final training, relies on early stopping
WEIGHT_DECAY = 1e-4
RANDOM_STATE = 42

# --- Uncertainty Parameters ---
MC_SAMPLES_FOR_CI = 100
EPSILON = 1e-8

# --- Setup ---
if torch:
    torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
device = torch.device("cuda" if torch and torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Enable optimizations if available
try:
    if torch:
        torch.backends.cudnn.benchmark = True
        if torch.cuda.is_available():
            if hasattr(torch.backends.cuda.matmul, 'allow_tf32'):
                 torch.backends.cuda.matmul.allow_tf32 = True
            if hasattr(torch, 'set_float32_matmul_precision'):
                torch.set_float32_matmul_precision("medium")
except Exception:
    pass

# =========================================================
#                 1.  Model definition (Seq2Seq/DMS)
# =========================================================
# Addressed Weakness 3: Switched to Direct Multi-Step (DMS) architecture.
class LSTMForecaster(nn.Module):
    """
    LSTM model adapted for Direct Multi-Step (DMS) forecasting.
    """
    def __init__(self, input_size: int, hidden_size: int,
                 output_size: int, dropout: float, num_layers: int):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )
        self.dropout = nn.Dropout(dropout)
        # Output layer maps the last hidden state to H forecast steps
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x shape: (batch, seq_len (lags), features)
        out, _ = self.lstm(x)
        # Use the hidden state of the last time step
        out = self.dropout(out[:, -1, :])
        # Output shape: (batch, FORECAST_HORIZON)
        return self.fc(out)

def build_model(hp: Dict, n_vars: int) -> nn.Module:
    mdl = LSTMForecaster(
        input_size=n_vars,
        hidden_size=int(hp["hidden_size"]),
        output_size=FORECAST_HORIZON, # Changed from 1 to H
        dropout=float(hp["dropout_rate"]),
        num_layers=int(hp["num_layers"]),
    ).to(device)
    try:
        # Optimization for modern GPUs (requires PyTorch >= 2.0)
        if hasattr(torch, 'compile'):
             return torch.compile(mdl, mode="reduce-overhead")
        return mdl
    except Exception:
        return mdl

# =========================================================
#                 2.  Utility functions
# =========================================================

# ---------- Metrics & Statistics ----------
def smape_loss_np(y_true, y_pred, eps: float = EPSILON) -> float:
    # Works for both (N,) and (N, H) shapes
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    num = np.abs(y_true - y_pred)
    den = (np.abs(y_true) + np.abs(y_pred) + eps) / 2.0
    # Mean over all dimensions (samples and horizon)
    return np.mean(num / den)

def calculate_metrics(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    insample: Optional[np.ndarray] = None,
):
    # Ensure inputs are flat if they are 1D vectors
    if y_true.ndim == 1 or (y_true.ndim > 1 and y_true.shape[1] == 1):
        y_true = y_true.ravel()
        y_pred = y_pred.ravel()

    smape = smape_loss_np(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + EPSILON))) * 100.0
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    mase = rmsse = np.nan

    # Use seasonal MASE/RMSSE as recommended by forecasting literature (Hyndman & Koehler, 2006)
    if insample is not None and len(insample) > SEASONAL_PERIOD:
        # MASE Denominator
        d = np.mean(np.abs(np.diff(insample, n=SEASONAL_PERIOD)))
        
        mae = np.mean(np.abs(y_true - y_pred))
        mse = np.mean((y_true - y_pred) ** 2)

        mase = mae / (d + EPSILON)
        
        # RMSSE Denominator
        denom_rmsse = np.mean((np.diff(insample, n=SEASONAL_PERIOD))**2)
        rmsse = np.sqrt(mse / (denom_rmsse + EPSILON))


    return {"SMAPE": smape, "MAPE": mape, "RMSE": rmse,
            "MASE": mase, "RMSSE": rmsse}

# Addressed Weakness 5: Statistical Significance Testing
def diebold_mariano_test(actuals: np.ndarray, pred1: np.ndarray, pred2: np.ndarray, horizon: int):
    """
    Implements the Diebold-Mariano Test (with Newey-West HAC estimator) 
    to compare predictive accuracy of two forecasts.
    """
    actuals = actuals.ravel()
    pred1 = pred1.ravel()
    pred2 = pred2.ravel()

    if len(actuals) != len(pred1) or len(actuals) != len(pred2):
        return np.nan, np.nan
    
    N = len(actuals)
    if N == 0:
        return np.nan, np.nan

    # Loss differential (using squared error loss)
    loss1 = (actuals - pred1)**2
    loss2 = (actuals - pred2)**2
    d = loss1 - loss2
    d_mean = np.mean(d)

    # Autocovariance function (Newey-West estimator for HAC robustness)
    # Bandwidth selection: typically related to the forecast horizon
    q = horizon 
    gamma = np.zeros(q + 1)
    
    # Calculate autocovariances
    for k in range(q + 1):
        if k == 0:
            gamma[k] = np.var(d, ddof=0)
        else:
            # Calculate autocovariance at lag k
            try:
                cov = np.cov(d[k:], d[:N-k], ddof=0)
                if cov.ndim == 2:
                    gamma[k] = cov[0, 1]
                else:
                    # Handle case where cov might return a scalar if input is constant/too short
                    gamma[k] = 0
            except Exception:
                gamma[k] = 0


    # Long-run variance (Newey-West)
    V = gamma[0] + 2 * np.sum([(1 - k/(q+1)) * gamma[k] for k in range(1, q+1)])

    if V <= 1e-9:
        # Handle case where variance is zero or near-zero
        if abs(d_mean) < 1e-9:
            return 0, 1.0 # Forecasts are identical
        else:
            return np.inf * np.sign(d_mean), 0.0

    # DM statistic (t-distributed under the null)
    DM_stat = d_mean / np.sqrt(V / N)

    # P-value (two-tailed test using t-distribution)
    p_value = 2 * (1 - t.cdf(np.abs(DM_stat), df=N-1))

    return DM_stat, p_value

# ---------- Smoothing (Kept as optional preprocessing) ----------
def fit_smoother(series, method, alpha=None, beta=None):
    if method == "NS":
        return None
    if SimpleExpSmoothing is None or Holt is None:
        # Only warn if a method was actually requested
        if method in ["ES", "DES"]:
             print("Warning: Smoothing methods requested but statsmodels components are missing.")
        return None

    try:
        if method == "ES":
            return SimpleExpSmoothing(series, initialization_method="estimated").fit(
                smoothing_level=alpha, optimized=False)
        if method == "DES":
            return Holt(series, initialization_method="estimated").fit(
                smoothing_level=alpha, smoothing_trend=beta, optimized=False)
    except Exception:
        # Handle potential fitting errors during HPO
        return None
    raise ValueError(f"Unknown smoothing method: {method}")

def apply_fitted_smoother(fitter, df: pd.DataFrame, target: str) -> pd.DataFrame:
    if fitter is None:
        return df
    n = len(df)
    fitted_vals = fitter.fittedvalues
    
    if len(fitted_vals) == 0:
        return df

    if len(fitted_vals) == n:
        values = fitted_vals
    elif len(fitted_vals) < n:
        # If fitted values are shorter (e.g. initialization period), pad with original data at the start
        # This implementation ensures the length matches the input dataframe.
        values = pd.concat([df[target].iloc[:n-len(fitted_vals)], fitted_vals])
    else:
        values = fitted_vals[:n]

    out = df.copy()
    
    # Ensure index alignment
    if len(values) == n:
        out[target] = values.values
    else:
        # Fallback if alignment fails unexpectedly
        return df 

    # Handle potential NaNs (though padding above should prevent this)
    out[target].bfill(inplace=True)
    return out

# ---------- Data Preparation (Major Update for DMS) ----------
def create_sequences_dms(df_in: pd.DataFrame, target: str,
                         features: List[str], num_lags: int, horizon: int) -> Tuple[np.ndarray, np.ndarray, List[str]]:
    """
    Creates input sequences (X) and target sequences (Y) for Direct Multi-Step forecasting.
    
    X: (N_samples, num_lags, N_features)
    Y: (N_samples, horizon)
    """
    cols = [target] + features
    data = df_in[cols].values
    N = data.shape[0]

    X, Y = [], []

    # Iterate through the data to create sequences
    for i in range(N - num_lags - horizon + 1):
        # Input sequence (X): from index i to i + num_lags
        x_seq = data[i : i + num_lags]
        # Target sequence (Y): from index i + num_lags to i + num_lags + horizon
        # We only want the target variable (index 0) for Y
        y_seq = data[i + num_lags : i + num_lags + horizon, 0]

        X.append(x_seq)
        Y.append(y_seq)

    return np.array(X), np.array(Y), cols


def select_mrmr_features(df_window: pd.DataFrame, target: str, k: int, num_lags: int) -> List[str]:
    """
    Selects features using mRMR based on the relationship with the immediate next target value (t+1).
    """
    if k == 0 or mrmr is None:
        return []

    # 1. Create temporary lagged dataframe for mRMR selection
    lagged = []
    potential_features = [c for c in df_window.columns if c != target]
    if not potential_features:
        return []

    # Create lags
    for col in [target] + potential_features:
        for l in range(1, num_lags + 1):
            lagged.append(df_window[col].shift(l).rename(f"{col}_lag_{l}"))

    df_lagged = pd.concat([df_window[[target]], *lagged], axis=1).dropna()

    if df_lagged.empty:
        return []

    # 2. Apply mRMR
    # Select the best *exogenous* features (excluding target's own lags from the selection pool)
    X_cols = [c for c in df_lagged.columns if "_lag_" in c and not c.startswith(f"{target}_lag_")]

    if not X_cols:
        return []

    X = df_lagged[X_cols].ffill().fillna(0)
    y = df_lagged[target].values.ravel()

    # Limit K to the number of available features
    K_eff = min(k, len(X_cols))

    try:
        # Use mrmr_regression for continuous target
        selected_lagged = mrmr.mrmr_regression(X=X, y=y, K=K_eff)
        # Get the base feature names
        selected_base = list({s.split("_lag_")[0] for s in selected_lagged})
        return selected_base
    except Exception as e:
        print(f"Warning: mRMR failed: {e}. Falling back to using no exogenous features.")
        return []

# ---------- Prediction Utilities ----------
def mc_pred(model: nn.Module, inp: torch.Tensor, mc_samples: int):
    """Generates Monte Carlo predictions if dropout is active."""
    # inp shape: (1, Lags, Features)
    if mc_samples == 1:
        model.eval() # Ensure eval mode if not doing MC dropout
        with torch.no_grad():
            # Output shape: (1, Horizon)
            return model(inp).cpu().numpy()

    # Enable dropout at inference time (MC Dropout)
    model.train() 
    with torch.no_grad():
        # Repeat input MC times. Shape: (MC_samples, Lags, Features)
        inp_mc = inp.repeat(mc_samples, 1, 1)
        # Output shape: (MC_samples, Horizon)
        preds = model(inp_mc).cpu().numpy()
    return preds


# =========================================================
#                 3.  Optuna objective (Updated for DMS)
# =========================================================
# Addressed Weakness 4: HPO now optimizes the 36-month forecast SMAPE.
def make_objective(tgt: str, train_val_df: pd.DataFrame,
                   k: int, mode: str):
    # Use TimeSeriesSplit for HPO validation. The test_size is the forecast horizon.
    tscv = TimeSeriesSplit(n_splits=N_CV_SPLITS, test_size=FORECAST_HORIZON)

    def objective(trial: optuna.Trial):
        hp = {
            # Architecture HPs (Expanded ranges)
            "hidden_size": trial.suggest_categorical("hidden_size", [64, 128, 256, 512]),
            "num_layers": trial.suggest_int("num_layers", 1, 3),
            "dropout_rate": trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1),
            # Training HPs
            "lr": trial.suggest_categorical("lr", [1e-4, 5e-4, 1e-3]),
            "batch_size": trial.suggest_categorical("batch_size", [32, 64, 128]),
            # Feature Engineering HPs (Crucial for sequence length)
            "n_lags": trial.suggest_int("n_lags", SEASONAL_PERIOD, SEASONAL_PERIOD * 4, step=SEASONAL_PERIOD),
            # Optional Preprocessing (Smoothing)
            "smoothing_method": trial.suggest_categorical("smoothing_method", ["NS", "ES", "DES"]),
            "smoothing_alpha": trial.suggest_float("smoothing_alpha", 0.1, 0.5),
            "smoothing_beta": trial.suggest_float("smoothing_beta", 0.05, 0.3),
        }

        n_lags = hp["n_lags"]
        smapes_cv = []

        # Iterate over Cross-Validation folds
        for tr_idx, val_idx in tscv.split(train_val_df):
            # Ensure enough data in training fold for lags and at least one training sample
            if len(tr_idx) < n_lags + FORECAST_HORIZON:
                continue

            df_tr = train_val_df.iloc[tr_idx]
            
            # 1. Preprocessing (Smoothing) - Fit only on training data
            smoother = fit_smoother(df_tr[tgt],
                                    hp["smoothing_method"],
                                    hp["smoothing_alpha"],
                                    hp["smoothing_beta"])
            df_tr_sm = apply_fitted_smoother(smoother, df_tr, tgt)

            # 2. Feature Selection (mRMR)
            if mode == "MULTI":
                # Select features based on the smoothed training data
                # Use a fixed lag length (e.g., 12) for the mRMR selection process itself
                base_feats = select_mrmr_features(df_tr_sm, tgt, k, num_lags=SEASONAL_PERIOD) 
            else:
                base_feats = []

            n_vars = len([tgt, *base_feats])

            # 3. Data Sequencing (DMS format)
            X_tr, Y_tr, _ = create_sequences_dms(df_tr_sm, tgt, base_feats, n_lags, FORECAST_HORIZON)

            # Prepare the validation input: The very last 'n_lags' of the training fold
            X_va_input = df_tr_sm[[tgt] + base_feats].iloc[-n_lags:].values.reshape(1, n_lags, n_vars)
            # Validation target: The actual 'FORECAST_HORIZON' points in the validation fold (ground truth)
            # We use the original (non-smoothed) data for validation targets.
            Y_va_target = train_val_df.iloc[val_idx][tgt].values.reshape(1, FORECAST_HORIZON)

            if X_tr.shape[0] == 0:
                continue # Not enough data for this fold/configuration

            # 4. Scaling
            # Reshape for MinMaxScaler
            X_tr_reshaped = X_tr.reshape(-1, n_vars)
            Y_tr_reshaped = Y_tr.reshape(-1, 1)

            sc_X = MinMaxScaler().fit(X_tr_reshaped)
            sc_y = MinMaxScaler().fit(Y_tr_reshaped)

            # Apply scaling and reshape back
            X_tr_scaled = sc_X.transform(X_tr.reshape(-1, n_vars)).reshape(X_tr.shape)
            Y_tr_scaled = sc_y.transform(Y_tr.reshape(-1, 1)).reshape(Y_tr.shape)
            X_va_scaled = sc_X.transform(X_va_input.reshape(-1, n_vars)).reshape(X_va_input.shape)

            # 5. Tensors and DataLoader
            Xtr_t = torch.tensor(X_tr_scaled, dtype=torch.float32)
            Ytr_t = torch.tensor(Y_tr_scaled, dtype=torch.float32)
            Xva_t = torch.tensor(X_va_scaled, dtype=torch.float32).to(device)

            dl = torch.utils.data.DataLoader(
                torch.utils.data.TensorDataset(Xtr_t, Ytr_t),
                batch_size=hp["batch_size"], shuffle=True)

            # 6. Model Training
            mdl = build_model(hp, n_vars)
            # L1Loss (MAE) is often robust for time series
            crit = nn.L1Loss() 
            opt = optim.AdamW(
                mdl.parameters(), lr=hp["lr"],
                weight_decay=WEIGHT_DECAY)

            best_fold_smape = float("inf")
            no_imp = 0

            for ep in range(MAX_EPOCHS_OPTUNA):
                mdl.train()
                for xb, yb in dl:
                    xb, yb = xb.to(device), yb.to(device)
                    opt.zero_grad(set_to_none=True)
                    # Mixed precision training
                    with torch.autocast(device.type, enabled=(device.type == "cuda")):
                        preds = mdl(xb)
                        # Loss is calculated over all H steps
                        loss = crit(preds, yb) 
                    
                    # Handle potential NaN loss during HPO
                    if torch.isnan(loss):
                        return float("inf")

                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(mdl.parameters(), 1.0)
                    opt.step()

                # 7. Validation (Crucial: Evaluate on the H-step forecast)
                mdl.eval()
                with torch.no_grad():
                    # Generate the H-step forecast
                    preds_scaled = mdl(Xva_t).cpu().numpy() # Shape (1, Horizon)
                    # Inverse transform
                    preds_inv = sc_y.inverse_transform(preds_scaled.reshape(-1, 1)).reshape(1, FORECAST_HORIZON)

                    # Calculate SMAPE against the ground truth validation target
                    s = smape_loss_np(Y_va_target, preds_inv)

                # Optuna pruning and Early Stopping logic
                trial.report(s, ep)
                if trial.should_prune():
                    raise optuna.TrialPruned()

                if s < best_fold_smape - 1e-4:
                    best_fold_smape = s
                    no_imp = 0
                else:
                    no_imp += 1

                if no_imp >= PATIENCE_ES:
                    break

            smapes_cv.append(best_fold_smape)

        if not smapes_cv:
            # If no folds completed (e.g., data too short for lags)
            return float("inf")

        # Return the average SMAPE across CV folds
        return float(np.mean(smapes_cv))

    return objective

# =========================================================
#                 4. HPO and Training Pipeline (Reworked)
# =========================================================
# Addressed Weakness 6 & 8: HPO via CV, Final training uses Early Stopping.

def train_final_model(hp: Dict, df_train: pd.DataFrame, tgt: str, base_feats: List[str], use_early_stopping: bool = True):
    """
    Trains the LSTM model with optimized hyperparameters.
    Includes early stopping on a validation subset of the training data.
    """
    n_lags = hp["n_lags"]
    n_vars = len([tgt, *base_feats])

    # 1. Preprocessing (Smoothing)
    smoother = fit_smoother(df_train[tgt],
                            hp["smoothing_method"],
                            hp["smoothing_alpha"],
                            hp["smoothing_beta"])
    df_train_sm = apply_fitted_smoother(smoother, df_train, tgt)

    # 2. Data Sequencing
    X, Y, _ = create_sequences_dms(df_train_sm, tgt, base_feats, n_lags, FORECAST_HORIZON)

    if X.shape[0] == 0:
        return None, None, None

    # 3. Splitting for Early Stopping
    # We reserve the most recent 20% of sequences for validation if ES is enabled.
    # This is a temporal split of the sequences.
    if use_early_stopping and X.shape[0] > 20:
         split_idx = int(X.shape[0] * 0.8)
         X_tr, Y_tr = X[:split_idx], Y[:split_idx]
         X_va, Y_va = X[split_idx:], Y[split_idx:]
    else:
        X_tr, Y_tr = X, Y
        X_va, Y_va = None, None

    # 4. Scaling (Fit only on the training split)
    X_tr_reshaped = X_tr.reshape(-1, n_vars)
    Y_tr_reshaped = Y_tr.reshape(-1, 1)

    sc_X = MinMaxScaler().fit(X_tr_reshaped)
    sc_y = MinMaxScaler().fit(Y_tr_reshaped)

    X_tr_scaled = sc_X.transform(X_tr.reshape(-1, n_vars)).reshape(X_tr.shape)
    Y_tr_scaled = sc_y.transform(Y_tr.reshape(-1, 1)).reshape(Y_tr.shape)

    Xtr_t = torch.tensor(X_tr_scaled, dtype=torch.float32)
    Ytr_t = torch.tensor(Y_tr_scaled, dtype=torch.float32)

    if X_va is not None:
        X_va_scaled = sc_X.transform(X_va.reshape(-1, n_vars)).reshape(X_va.shape)
        Y_va_scaled = sc_y.transform(Y_va.reshape(-1, 1)).reshape(Y_va.shape)
        Xva_t = torch.tensor(X_va_scaled, dtype=torch.float32).to(device)
        Yva_t = torch.tensor(Y_va_scaled, dtype=torch.float32).to(device)
    else:
        Xva_t, Yva_t = None, None

    dl = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(Xtr_t, Ytr_t),
        batch_size=hp["batch_size"], shuffle=True)

    # 5. Model Initialization
    mdl = build_model(hp, n_vars)
    crit = nn.L1Loss()
    opt = optim.AdamW(
        mdl.parameters(), lr=hp["lr"],
        weight_decay=WEIGHT_DECAY)

    # 6. Training Loop with Early Stopping
    best_val_loss = float("inf")
    no_imp = 0
    best_model_state = None
    
    # Initialize best state if no validation set is used (we will use the final weights)
    if Xva_t is None:
         best_model_state = copy.deepcopy(mdl.state_dict())


    for ep in range(MAX_EPOCHS_FINAL):
        mdl.train()
        for xb, yb in dl:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad(set_to_none=True)
            with torch.autocast(device.type, enabled=(device.type == "cuda")):
                preds = mdl(xb)
                loss = crit(preds, yb)
            
            if torch.isnan(loss):
                # If loss becomes NaN, stop training and return None
                return None, None, None

            loss.backward()
            torch.nn.utils.clip_grad_norm_(mdl.parameters(), 1.0)
            opt.step()

        # Validation phase for Early Stopping
        if Xva_t is not None:
            mdl.eval()
            with torch.no_grad():
                 with torch.autocast(device.type, enabled=(device.type == "cuda")):
                    val_preds = mdl(Xva_t)
                    val_loss = crit(val_preds, Yva_t).item()

            if val_loss < best_val_loss - 1e-4:
                best_val_loss = val_loss
                no_imp = 0
                # Save best model weights
                best_model_state = copy.deepcopy(mdl.state_dict())
            else:
                no_imp += 1

            if no_imp >= PATIENCE_ES:
                # print(f"Early stopping triggered at epoch {ep+1}/{MAX_EPOCHS_FINAL}")
                break
    
    # Load the best weights found during training
    if best_model_state:
         mdl.load_state_dict(best_model_state)

    return mdl, sc_X, sc_y


def run_hpo_pipeline(df_dev: pd.DataFrame, targets: List[str], k: int, mode: str) -> Dict:
    """
    Runs the Hyperparameter Optimization pipeline for all targets on the development set.
    """
    best_cfgs = {}
    print(f"\n--- Starting HPO Pipeline (Mode: {mode}) ---")

    for tgt in targets:
        print(f"\nOptimizing Target: {tgt}")

        # Define the study
        study_name = f"{mode}_{tgt}_HPO_DMS"
        try:
            study = optuna.create_study(
                study_name=study_name,
                direction="minimize", # Minimize H-month SMAPE
                sampler=TPESampler(seed=RANDOM_STATE),
                pruner=SuccessiveHalvingPruner(min_resource=4, reduction_factor=3),
                storage=STORAGE, load_if_exists=True)
        except Exception as e:
             print(f"Could not create/load Optuna study: {e}")
             continue

        # Run optimization
        if len(study.trials) < OPTUNA_N_TRIALS:
            objective_fn = make_objective(tgt, df_dev, k, mode)
            try:
                study.optimize(objective_fn,
                               n_trials=OPTUNA_N_TRIALS - len(study.trials),
                               n_jobs=1, # Keep n_jobs=1 for stability with PyTorch/GPU
                               show_progress_bar=True)
            except Exception as e:
                 print(f"Optuna optimization failed for {tgt}: {e}")
                 continue


        if study.best_trial is None:
            print(f"HPO failed for {tgt}. Skipping.")
            continue

        print(f"Best CV SMAPE for {tgt}: {study.best_value:.4f}")
        
        best_hp = study.best_params
        
        # Determine the final feature set by running mRMR on the full development set
        # using the optimized preprocessing parameters.
        if mode == "MULTI":
            smoother = fit_smoother(df_dev[tgt],
                                    best_hp.get("smoothing_method", "NS"),
                                    best_hp.get("smoothing_alpha", 0.5),
                                    best_hp.get("smoothing_beta", 0.1))
            df_dev_sm = apply_fitted_smoother(smoother, df_dev, tgt)
            # Use a standard lag (e.g., 12) for the feature selection process
            final_feats = select_mrmr_features(df_dev_sm, tgt, k, num_lags=SEASONAL_PERIOD)
        else:
            final_feats = []

        best_cfgs[tgt] = {
            "smape_dev": study.best_value,
            "hyperparams": best_hp,
            "base_features": final_feats,
            "n_vars_per_step": len([tgt, *final_feats]),
            "n_lags": best_hp["n_lags"]
        }

    return best_cfgs

# =========================================================
#                 5.  Forecast & Evaluation
# =========================================================

def generate_forecast_dms(
    df_hist: pd.DataFrame, target: str, cfg: Dict, horizon: int,
    mc_samples: int):
    """
    Trains the model on historical data and generates a H-step ahead forecast with CI.
    """
    if not cfg or not cfg.get("hyperparams"):
        return None

    hp = cfg["hyperparams"]
    feats = cfg["base_features"]
    n_lags = cfg["n_lags"]
    n_vars = cfg["n_vars_per_step"]

    if horizon != FORECAST_HORIZON:
         raise ValueError(f"Model architecture fixed to horizon {FORECAST_HORIZON}, cannot forecast {horizon}.")

    # 1. Train the model on the full historical data (df_hist)
    mdl, sc_X, sc_y = train_final_model(hp, df_hist, target, feats, use_early_stopping=True)

    if mdl is None:
        return None

    # 2. Prepare the input for forecasting (the last 'n_lags' points)

    # We must apply the same preprocessing (smoothing) to the historical data 
    # before creating the final input sequence. This ensures consistency.
    smoother = fit_smoother(df_hist[target],
                            hp["smoothing_method"],
                            hp["smoothing_alpha"],
                            hp["smoothing_beta"])
    df_hist_sm = apply_fitted_smoother(smoother, df_hist, target)

    input_data = df_hist_sm[[target] + feats].iloc[-n_lags:].values
    
    # Scale the input using the scaler fitted during the final training
    input_scaled = sc_X.transform(input_data.reshape(-1, n_vars))
    # Reshape for model input: (1, Lags, Features)
    inp_t = torch.tensor(input_scaled.reshape(1, n_lags, n_vars),
                         dtype=torch.float32, device=device)

    # 3. Generate Forecasts (with MC Dropout for Uncertainty)
    # Use MC samples if requested AND if the model has dropout > 0.
    use_mc = mc_samples > 1 and hp.get("dropout_rate", 0) > 0
    samples = mc_samples if use_mc else 1

    # preds_mc shape: (Samples, Horizon)
    preds_mc_scaled = mc_pred(mdl, inp_t, samples)

    # Inverse transform
    preds_inv = sc_y.inverse_transform(preds_mc_scaled.reshape(-1, 1)).reshape(samples, horizon)

    # 4. Calculate point forecast and CIs (95% CI)
    if use_mc:
        # Use median as the point forecast (robust)
        forecasts = np.median(preds_inv, axis=0)
        lowers = np.percentile(preds_inv, 2.5, axis=0)
        uppers = np.percentile(preds_inv, 97.5, axis=0)
    else:
        forecasts = preds_inv[0]
        # CIs are NaN if MC Dropout is not used
        lowers = np.full(horizon, np.nan)
        uppers = np.full(horizon, np.nan)

    # Optional: Ensure non-negative forecasts if the domain requires it
    forecasts[forecasts < 0] = 0
    if use_mc:
        lowers[lowers < 0] = 0
        uppers[uppers < 0] = 0

    return pd.DataFrame({"Forecast": forecasts,
                         "Lower_CI": lowers, "Upper_CI": uppers})

# ---------- Baseline Models (Weakness 2 Addressed) ----------

def forecast_arima(series: pd.Series, horizon: int):
    if pm is None:
        # pmdarima failed to import
        return np.full(horizon, np.nan)
    try:
        # Rigorous Auto-ARIMA implementation
        model = pm.auto_arima(series,
                              start_p=1, start_q=1,
                              max_p=5, max_q=5,
                              m=SEASONAL_PERIOD,
                              start_P=0, seasonal=True,
                              d=None, D=1, trace=False,
                              error_action='ignore',  
                              suppress_warnings=True, 
                              stepwise=True)
        forecast = model.predict(n_periods=horizon)
        return np.asarray(forecast)
    except Exception as e:
        # ARIMA can fail if the series is constant, too short, or optimization fails
        # print(f"ARIMA failed: {e}")
        return np.full(horizon, np.nan)

def forecast_ets(series: pd.Series, horizon: int):
    if ETSModel is None:
        return np.full(horizon, np.nan)
    try:
        # Optimized ETS implementation (Automatic model selection)
        # We try additive components first as they are generally more robust.
        model = ETSModel(series, error="add", trend="add", seasonal="add",
                         damped_trend=True, seasonal_periods=SEASONAL_PERIOD)
        fit = model.fit(disp=False, optimized=True)
        forecast = fit.forecast(horizon)
        return np.asarray(forecast)
    except Exception as e:
        # print(f"ETS failed: {e}")
        return np.full(horizon, np.nan)


# =========================================================
#                 6.  Main Execution
# =========================================================
if __name__ == "__main__":
    # Set appropriate warning levels
    if ConvergenceWarning:
        warnings.simplefilter("ignore", ConvergenceWarning)
    if ValueWarning:
        warnings.simplefilter("ignore", ValueWarning)
    warnings.simplefilter("ignore", UserWarning)
    
    # Reduce verbosity for cleaner output during HPO
    if optuna:
        try:
            optuna.logging.set_verbosity(optuna.logging.WARNING)
        except Exception:
            pass

    # --- Environment Logging ---
    env_info = {
        "timestamp_utc": dt.datetime.utcnow().isoformat() + "Z",
        "torch": torch.__version__ if torch else "N/A",
        "FORECAST_HORIZON": FORECAST_HORIZON,
    }
    if torch and torch.cuda.is_available():
        try:
            env_info["device"] = torch.cuda.get_device_name(0)
        except Exception:
            env_info["device"] = "CUDA (Name Unavailable)"
    else:
        env_info["device"] = "CPU"
        
    try:
        json.dump(env_info, open(BASE_OUT / "env.json", "w"), indent=2)
    except Exception as e:
        print(f"Could not write env.json: {e}")

    # ------------- Load & Clean Data -------------
    ROOT / "yourdatahere"
    
    # !!! DATA LOADING !!!
    # Replace this MOCK DATA section with your actual data loading if running locally:
    if DF_PATH.exists():
        df_raw = pd.read_excel(DF_PATH)
    else: 
         print(f"Data file not found at {DF_PATH}. Proceeding with MOCK DATA.")
    



    date_col = next((c for c in df_raw.columns
                     if c.lower().strip() == "date"), None)
    if date_col:
        df_raw[date_col] = pd.to_datetime(df_raw[date_col])
        df_raw.sort_values(date_col, inplace=True)
        df_raw.set_index(date_col, inplace=True)

    df_raw.columns = df_raw.columns.str.lower().str.strip()
    
    # Data Cleaning (Numeric conversion and imputation)
    for col in df_raw.columns:
        if df_raw[col].dtype == "object":
            try:
                df_raw[col] = pd.to_numeric(
                    df_raw[col].str.replace(r"[^\d.\-]", "", regex=True),
                    errors="coerce")
            except AttributeError:
                # Handle cases where it's not a string but still object type (e.g., mixed types)
                df_raw[col] = pd.to_numeric(df_raw[col], errors="coerce")

    # Imputation: ffill is reasonable. Filling initial NaNs with 0 must be justified by the domain.
    df_raw = df_raw.ffill().fillna(0)

    # ------------- Data Splitting -------------
    # Rigorous split: Development (Train+Val) and Test (Hold-out)
    # The test set size MUST equal the forecast horizon.
    df_dev = df_raw.iloc[:-FORECAST_HORIZON].copy()
    df_test = df_raw.iloc[-FORECAST_HORIZON:].copy()

    if len(df_dev) < FORECAST_HORIZON * 3:
         print("Warning: Development set might be short relative to the horizon, potentially impacting CV stability.")

    # Define targets
    targets_all = [
        "generative ai journal article",
        "autonomous vehicles journal article",
        "digital twins journal article",
        "misinformation journal article",
        "3d printing journal article", "new programming models journal article",
        "renewable energy journal article",
        "sustainable technologies journal article", "generative agritech journal article",
        "metaverse journal article",
    ]
    # Filter targets present in the dataframe
    targets = [t.lower().strip() for t in targets_all
               if t.lower().strip() in df_dev.columns]
    
    if not targets:
        # Fallback for mock data if the exact names aren't used
        targets = [col for col in df_raw.columns if 'journal article' in col]
        if not targets:
             raise RuntimeError("No valid targets found in data.")

    # ------------- HPO Execution -------------
    K_FEATURES = 10 # Max exogenous features for mRMR
    best_cfgs_modes = {}

    # Run Univariate and Multivariate pipelines
    for MODE in ["UNI", "MULTI"]:
        if MODE == "MULTI" and (mrmr is None or len(df_raw.columns) < 2):
            print(f"Skipping {MODE} mode (mRMR missing or insufficient features).")
            continue
            
        # Run HPO
        best_cfgs = run_hpo_pipeline(df_dev, targets, k=K_FEATURES, mode=MODE)
        best_cfgs_modes[MODE] = best_cfgs

        # Save HPO results
        best_path = BASE_OUT / f"best_hyperparameters_{MODE}.json"
        try:
            # Atomic write
            tmp = best_path.with_suffix(".tmp")
            with open(tmp, 'w') as f:
                # Use a custom JSON encoder to handle potential NumPy types if any slipped in
                class NumpyEncoder(json.JSONEncoder):
                    def default(self, obj):
                        if isinstance(obj, np.integer):
                            return int(obj)
                        elif isinstance(obj, np.floating):
                            return float(obj)
                        elif isinstance(obj, np.ndarray):
                            return obj.tolist()
                        return super(NumpyEncoder, self).default(obj)

                json.dump(best_cfgs, f, indent=2, cls=NumpyEncoder)
            tmp.replace(best_path)
        except Exception as e:
            print(f"Could not save HPO results for {MODE}: {e}")

    # ------------- Test Set Evaluation and Baseline Comparison -------------
    print("\n--- Starting Test Set (Hold-out) Evaluation ---")
    test_results = []
    baseline_cache = {} # Cache baselines as they are the same for UNI/MULTI

    for MODE in best_cfgs_modes.keys():
        best_cfgs = best_cfgs_modes[MODE]

        for tgt in targets:
            # print(f"\nEvaluating {MODE} LSTM for: {tgt}")
            cfg = best_cfgs.get(tgt)
            if not cfg:
                continue

            # 1. LSTM Forecast on Test Set
            # Train on df_dev, predict the horizon corresponding to df_test
            # Use mc_samples=1 for point forecast evaluation metrics
            fc_df = generate_forecast_dms(
                df_dev, tgt, cfg, FORECAST_HORIZON, mc_samples=1)

            if fc_df is None:
                print(f"Skipping {tgt} ({MODE}) due to model training failure.")
                continue

            y_true = df_test[tgt].values.ravel()
            y_pred_lstm = fc_df["Forecast"].values.ravel()
            insample_data = df_dev[tgt].values

            lstm_m = calculate_metrics(y_true, y_pred_lstm, insample_data)
            result_row = {
                "Target": tgt, "Mode": MODE,
                **{f"LSTM_{k}": v for k, v in lstm_m.items()}
            }

            # 2. Baselines (Weakness 2 Addressed)
            if tgt not in baseline_cache:
                print(f"Calculating Baselines for {tgt} (Naive, SNaive, ARIMA, ETS)...")
                
                # Naive (Persistence)
                naive_pred = np.repeat(insample_data[-1], FORECAST_HORIZON)
                
                # Seasonal Naive
                if len(insample_data) >= SEASONAL_PERIOD:
                    snaive_pred = np.tile(insample_data[-SEASONAL_PERIOD:],
                                          math.ceil(FORECAST_HORIZON/SEASONAL_PERIOD))[:FORECAST_HORIZON]
                else:
                    snaive_pred = naive_pred # Fallback

                # Auto-ARIMA (returns NaN if pm is None)
                arima_pred = forecast_arima(df_dev[tgt], FORECAST_HORIZON)

                # ETS (returns NaN if ETSModel is None)
                ets_pred = forecast_ets(df_dev[tgt], FORECAST_HORIZON)

                baseline_cache[tgt] = {
                    "Naive": naive_pred,
                    "SNaive": snaive_pred,
                    "ARIMA": arima_pred,
                    "ETS": ets_pred,
                }

            # Calculate metrics for baselines and add to results
            for name, pred in baseline_cache[tgt].items():
                 # Check if the baseline generated valid predictions
                 if np.isnan(pred).all():
                     metrics = {"SMAPE": np.nan, "MAPE": np.nan, "RMSE": np.nan, "MASE": np.nan, "RMSSE": np.nan}
                 else:
                    metrics = calculate_metrics(y_true, pred, insample_data)
                 result_row.update({f"{name}_{k}": v for k, v in metrics.items()})

            # 3. Statistical Significance (Weakness 5 Addressed)
            # Compare LSTM against the best statistical model (ARIMA or ETS)
            
            arima_smape = result_row.get("ARIMA_SMAPE", np.inf)
            ets_smape = result_row.get("ETS_SMAPE", np.inf)
            
            # Handle cases where SMAPE might be NaN if the model failed
            if np.isnan(arima_smape): arima_smape = np.inf
            if np.isnan(ets_smape): ets_smape = np.inf

            # Determine the best performing statistical baseline
            best_stat_name = None
            if arima_smape != np.inf or ets_smape != np.inf:
                if arima_smape <= ets_smape:
                    best_stat_name = "ARIMA"
                    best_stat_pred = baseline_cache[tgt]["ARIMA"]
                else:
                    best_stat_name = "ETS"
                    best_stat_pred = baseline_cache[tgt]["ETS"]

            if best_stat_name:
                # Check if predictions are valid (non-NaN) before running the test
                if np.isnan(y_pred_lstm).any() or np.isnan(best_stat_pred).any():
                    dm_stat, dm_p = np.nan, np.nan
                else:
                    # Run Diebold-Mariano test
                    dm_stat, dm_p = diebold_mariano_test(y_true, y_pred_lstm, best_stat_pred, FORECAST_HORIZON)

                result_row[f"DM_vs_{best_stat_name}_Stat"] = dm_stat
                result_row[f"DM_vs_{best_stat_name}_Pval"] = dm_p

            test_results.append(result_row)

    # Consolidate and save test results
    df_results = pd.DataFrame(test_results)
    
    # Restructure the results table for easier comparison (One row per target, columns for models)
    if not df_results.empty:
        try:
            if len(df_results["Mode"].unique()) > 1:
                modes = list(df_results["Mode"].unique())
                
                # Assume the first mode calculated the baselines
                df_base = df_results[df_results["Mode"] == modes[0]].drop(columns="Mode")
                
                for mode in modes[1:]:
                    df_mode = df_results[df_results["Mode"] == mode]
                    # Keep only LSTM metrics and DM test results for subsequent modes
                    lstm_cols = [col for col in df_mode.columns if col.startswith("LSTM_") or col.startswith("DM_") or col == "Target"]
                    df_mode = df_mode[lstm_cols]
                    # Rename columns to indicate the mode (e.g., LSTM_SMAPE -> LSTM_MULTI_SMAPE)
                    rename_dict = {c: c.replace("LSTM_", f"LSTM_{mode}_") for c in lstm_cols if c.startswith("LSTM_")}
                    # Handle DM renaming specifically to keep the baseline name
                    rename_dm = {}
                    for c in lstm_cols:
                        if c.startswith("DM_"):
                             # e.g., DM_vs_ARIMA_Pval -> DM_MULTI_vs_ARIMA_Pval
                             rename_dm[c] = c.replace("DM_vs_", f"DM_{mode}_vs_")
                    
                    df_mode = df_mode.rename(columns=rename_dict, inplace=False)
                    df_mode = df_mode.rename(columns=rename_dm, inplace=False)

                    
                    df_base = pd.merge(df_base, df_mode, on="Target", how="outer")
                
                df_final_results = df_base
            else:
                 df_final_results = df_results.drop(columns="Mode", errors='ignore')

            df_final_results.to_csv(BASE_OUT / "test_evaluation_summary.csv", index=False)
            print("\nTest Evaluation Summary saved to test_evaluation_summary.csv")

        except Exception as e:
            print(f"Error restructuring results table: {e}")
            df_results.to_csv(BASE_OUT / "test_evaluation_raw.csv", index=False)


    # ------------- Long-Term Forecast (Future Projection) -------------
    print("\n--- Generating Final 36-Month Future Forecasts (using full data) ---")
    # Train on the full dataset (df_raw) and forecast the subsequent 36 months.
    df_full = pd.concat([df_dev, df_test])

    for MODE, best_cfgs in best_cfgs_modes.items():
        for tgt in targets:
            cfg = best_cfgs.get(tgt)
            if not cfg:
                continue
                
            # print(f"Projecting Future for {tgt} ({MODE})...")

            # Generate forecast with Confidence Intervals using MC Dropout
            fc_df = generate_forecast_dms(
                df_full, tgt, cfg, FORECAST_HORIZON,
                MC_SAMPLES_FOR_CI)

            if fc_df is None:
                print(f"Skipping future projection for {tgt} ({MODE}) due to training failure.")
                continue

            # Save the forecast
            safe_name = "".join(c if c.isalnum() else "_" for c in tgt)
            fc_df.to_csv(BASE_OUT / f"forecast_future_36m_{MODE.lower()}_{safe_name}.csv", index=False)

    print(f"\nAll tasks finished successfully. Outputs are located at: {BASE_OUT}")


Reason: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

ACTION REQUIRED: This is due to NumPy binary incompatibility.
To fix, run in your terminal: pip install --upgrade numpy && pip install --upgrade --force-reinstall scipy statsmodels pmdarima

Using device: cuda

--- Starting HPO Pipeline (Mode: UNI) ---

Optimizing Target: quantum computing journal article


  0%|          | 0/50 [00:00<?, ?it/s]

Best CV SMAPE for quantum computing journal article: 0.2030

Optimizing Target: generative ai journal article


  0%|          | 0/50 [00:00<?, ?it/s]

Best CV SMAPE for generative ai journal article: 0.2144

Optimizing Target: llm journal article


...

Best CV SMAPE for next generation ai journal article: 0.1833


100%|██████████| 10/10 [00:17<00:00,  1.76s/it]


Optimizing Target: advances in cybersecurity journal article


  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 10/10 [00:14<00:00,  1.49s/it]

100%|██████████| 10/10 [00:16<00:00,  1.64s/it]

100%|██████████| 10/10 [00:16<00:00,  1.67s/it]

100%|██████████| 10/10 [00:17<00:00,  1.70s/it]

 40%|████      | 4/10 [00:05<00:08,  1.36s/it]